In [ ]:
    
    # Step 7: Additional Catalog Corrections    
    
    # TO DO: Check what filters are present, we want all info regardless if all filters are present. 
    
    #Merge Each Filter into One Catalog 
    
    #Try only keeping things where errors are good.
    by_filter['UVM2'] = by_filter['UVM2'][by_filter['UVM2'].UVM2_MAG_ERR < 0.35]
    by_filter['UVW1'] = by_filter['UVW1'][by_filter['UVW1'].UVW1_MAG_ERR < 0.35]
    by_filter['UVW2'] = by_filter['UVW2'][by_filter['UVW2'].UVW2_MAG_ERR < 0.35]

    #Orignial Stuff:
    first_two = pd.merge(by_filter['UVM2'],by_filter['UVW2'],on=['Ra','Dec'],how='outer')
    catalog = pd.merge(first_two,by_filter['UVW1'],on=['Ra','Dec'],how='outer')

    #  Remove duplicate optical photometry
    catalog = catalog.drop(labels=['Umag_y', 'e_Umag_y', 'Bmag_y', 'e_Bmag_y', 'Vmag_y',
                                   'e_Vmag_y', 'Imag_y', 'e_Imag_y', 'Flag_y', 'Jmag_y', 
                                   'e_Jmag_y','Hmag_y', 'e_Hmag_y', 'Ksmag_y', 'e_Ksmag_y',
                                   'Umag_x', 'e_Umag_x', 'Bmag_x', 'e_Bmag_x', 'Vmag_x',
                                   'e_Vmag_x', 'Imag_x', 'e_Imag_x', 'Flag_x', 'Jmag_x', 
                                   'e_Jmag_x','Hmag_x', 'e_Hmag_x', 'Ksmag_x', 'e_Ksmag_x'],
                          axis=1)

    # Might as well not keep things with no photometry 
    catalog = catalog[(np.isfinite(catalog.UVM2_RA)) | (np.isfinite(catalog.UVW1_RA)) | (np.isfinite(catalog.UVW2_RA)) ]
    catalog.to_csv(f"{path}/{observation_id}/{observation_id}_1_1.csv")

    # Step 8 Plot Tractor Results 

    f,axes=plt.subplots(3,3,figsize=(20,20))
    fontsize = 20
    for i,h in enumerate(hdu):
        uvfilter = h.header['FILTER']
    
        data = TractorObjects[uvfilter].image
        model = TractorObjects[uvfilter].model
        residual = data - model 

        axes[i,0].imshow(data,vmin=vmin(data),vmax=vmax(data),origin="lower")
        axes[i,1].imshow(model,vmin=vmin(data),vmax=vmax(data),origin="lower")
        axes[i,2].imshow(residual,vmin=vmin(residual),vmax=vmax(residual),origin="lower")
    
        axes[i,0].set_ylabel(uvfilter,fontsize=fontsize)


    plt.savefig(f"{path}/{observation_id}/{observation_id}_1_1_tractor.png")
    #plt.close()  


    # Step 9 Plot Photometry 

    #plt.style.use('bmh')
    
    g,axs=plt.subplots(2,4,figsize=(30,20))

    uvm2_v = catalog[(catalog.UVM2_MAG_ERR < 0.35) & (catalog.UVM2_SATURATED == False) & (catalog.UVM2_SSS == 1.0) & (catalog.UVM2_EDGE == 1.0) & (catalog.e_Vmag < 0.35)]
    uvw1_v = catalog[(catalog.UVW1_MAG_ERR < 0.35) & (catalog.UVW1_SATURATED == False) & (catalog.UVW1_SSS == 1.0) & (catalog.UVW1_EDGE == 1.0) & (catalog.e_Vmag < 0.35)]
    uvw2_v = catalog[(catalog.UVW2_MAG_ERR < 0.35) & (catalog.UVW2_SATURATED == False) & (catalog.UVW2_SSS == 1.0) & (catalog.UVW2_EDGE == 1.0) & (catalog.e_Vmag < 0.35)]

    uvm2_uvw1 = catalog[(catalog.UVM2_MAG_ERR < 0.35) & (catalog.UVM2_SATURATED == False) & (catalog.UVM2_SSS == 1.0) & 
                        (catalog.UVM2_EDGE == 1.0) & (catalog.UVW1_MAG_ERR < 0.35) & (catalog.UVW1_SATURATED == False) & 
                        (catalog.UVW1_SSS == 1.0) & (catalog.UVW1_EDGE == 1.0)]

    axs[0,0].scatter(uvw1_v.UVW1_MAG - uvw1_v.Vmag,uvw1_v.UVW1_MAG)
    axs[0,1].scatter(uvm2_v.UVM2_MAG - uvm2_v.Vmag,uvm2_v.UVM2_MAG )
    axs[0,2].scatter(uvw2_v.UVW2_MAG - uvw2_v.Vmag,uvw2_v.UVW2_MAG)
    axs[0,3].scatter(uvm2_uvw1.UVM2_MAG - uvm2_uvw1.UVW1_MAG,uvm2_uvw1.UVM2_MAG)

    axs[1,0].scatter(uvw1_v.UVW1_MAG - uvw1_v.Vmag,uvw1_v.Vmag)
    axs[1,1].scatter(uvm2_v.UVM2_MAG - uvm2_v.Vmag,uvm2_v.Vmag )
    axs[1,2].scatter(uvw2_v.UVW2_MAG - uvw2_v.Vmag,uvw2_v.Vmag)
    axs[1,3].scatter(uvm2_uvw1.UVM2_MAG - uvm2_uvw1.UVW1_MAG,uvm2_uvw1.UVW1_MAG)

    axs[0,0].set_xlabel("UVW1 - V",fontsize=fontsize); axs[0,0].set_ylabel("UVW1",fontsize=fontsize);
    axs[0,1].set_xlabel("UVM2 - V",fontsize=fontsize); axs[0,1].set_ylabel("UVM2",fontsize=fontsize);
    axs[0,2].set_xlabel("UVW2 - V",fontsize=fontsize); axs[0,2].set_ylabel("UVW2",fontsize=fontsize);
    axs[0,3].set_xlabel("UVM2 - UVW1",fontsize=fontsize); axs[0,3].set_ylabel("UVM2",fontsize=fontsize);

    axs[1,0].set_xlabel("UVW1 - V",fontsize=fontsize); axs[1,0].set_ylabel("V",fontsize=fontsize);
    axs[1,1].set_xlabel("UVM2 - V",fontsize=fontsize); axs[1,1].set_ylabel("V",fontsize=fontsize);
    axs[1,2].set_xlabel("UVW2 - V",fontsize=fontsize); axs[1,2].set_ylabel("V",fontsize=fontsize);
    axs[1,3].set_xlabel("UVM2 - UVW1",fontsize=fontsize); axs[1,3].set_ylabel("UVW1",fontsize=fontsize);

    [ax.set_ylim(19,11) for ax in axs[0]]
    [ax.set_xlim(-7,4) for ax in axs[0]]

    [ax.set_ylim(20,12) for ax in axs[1]]
    [ax.set_xlim(-7,4) for ax in axs[1]]


    plt.savefig(f"{path}/{observation_id}/{observation_id}_1_1_photometry.png")
    #plt.close()  

    # Step 10 Plot Coordinates 
    plt.figure(figsize=(10,10))

    d = meta['UVM2'].data
    x,y = meta['UVM2'].pixel_positions

    plt.imshow(d,vmin=vmin(d),vmax=vmax(d),origin="lower")
    plt.scatter(x,y,s=5,c='red')

    plt.xlim(500,600)
    plt.ylim(500,600)

    plt.savefig(f"{path}/{observation_id}/{observation_id}_1_1coordinates.png")

    print("Total Time for ObsID: ",observation_id,' (1_1) ', (time.time()-t1)/60)    